In [1]:
import os
import sys
sys.path.insert(1, '../../Src/')
import numpy as np
from utils import preprocessing
from models import training
from models import models
from utils import make_report

PATH_DATA          = '../../Data/'
PATH_MODEL         = '../../Results/Models/'
PATH_RESULTS = '../../Results/'


FNMAME             = "stringer_spontaneous.npy"
FEATURES           = "sresp"


THRESHOLDING_FLAG  = False
THRESHOLDING_MAX   = 95  # 95%


NORMALIZATION_FLAG = True
TEST_SIZE          = 0.2

PCA_FLAG           = True



N_SPLITS = 5


np.random.seed(42)

%matplotlib inline
%load_ext autoreload 
%autoreload 2

In [2]:
targets = ["run"]#,"pupilArea","pupilCOM_x","pupilCOM_y"]  #  
variances = [0.75]#[0.25,0.5,0.75,0.9]

In [3]:
######################
for TARGET in targets:
    print('*'*20, TARGET,'*'*20)
    # make processing
    X,y = preprocessing.get_dataset(PATH_DATA,FNMAME,FEATURES,TARGET)

    X_train,X_test,y_train,y_test = preprocessing.simple_temporal_splitting(X,y,test_size=TEST_SIZE)
    
    y_train = y_train.astype(np.float32)
    
    mean_train = np.mean(y_train[1-np.isnan(y_train)] )
    
    if np.sum(np.isnan(y_train))>0:
        print(' nulls in y_train target :',np.sum(np.isnan(y_train)))
        y_train[np.isnan(y_train)] = mean_train

    if np.sum(np.isnan(y_test))>0:
        print(' nulls in y_test target :',np.sum(np.isnan(y_test)))
        y_test[np.isnan(y_test)]   = mean_train


    if THRESHOLDING_FLAG:
        X_train,X_test = preprocessing.thresholding(X_train,X_test,thresholing_max=THRESHOLDING_MAX)

    if NORMALIZATION_FLAG:
        X_train,X_test = preprocessing.normalization(X_train,X_test)
    

    for MAX_VARIANCE_EXPLAINED in variances:
        
        if PCA_FLAG:
            #MAX_VARIANCE_EXPLAINED
            X_train_pca,X_test_pca = preprocessing.auto_PCA(X_train,X_test,max_variance_explanation=None,max_components=20)
        
        
        N_DIMENSION = X_train_pca.shape[1]
        
        training.training_model_cv(X_train_pca,y_train,
                                          X_test_pca,  y_test,
                                          n_splits = N_SPLITS,
                                          total_models=models.total_models,
                                          path_model=PATH_MODEL)
        print('report starting..')        
        make_report.make_report(TARGET,PATH_RESULTS,PATH_MODEL,SUFFIX='dim_'+str(N_DIMENSION)+'_var_'+str(MAX_VARIANCE_EXPLAINED))
        print('TARGET = ',TARGET,'MAX_VARIANCE_EXPLAINED = ',MAX_VARIANCE_EXPLAINED,' report completed!')

******************** run ********************
dict_keys(['sresp', 'run', 'beh_svd_time', 'beh_svd_mask', 'stat', 'pupilArea', 'pupilCOM', 'xyz'])
 X size : (7018, 11983)
 y size : (7018, 1)
X train :  (5264, 11983) y train : (5264, 1)
X test  :  (1754, 11983) y test  : (1754, 1)
Normalization process ..
X train : mean [0]= 6.6743774 std [0]= 6.6743774
X test  : mean [0]= 7.5208654 std [0]= 8.151653
Normalization completed!
X train : mean [0]= 1.9628543e-07 std [0]= 1.9628543e-07
X test  : mean [0]= 0.1156612 std [0]= 1.1138103
PCA processing ..
X_train size : (5264, 11983)
X_test size  : (1754, 11983)
# principal components : 5264
PCA completed!
max variance explained : 1.0
X_train_pca size : (5264, 5264)
X_test_pca size  : (1754, 5264)
**********  Training  LinearRegression **********
*****  Fold  1 *****
Train fold : RMSE = 2e-05 R2 = 1.0
Val  fold  : RMSE = 3.43699 R2 = 0.81204
Test total : RMSE = 4.00594 R2 = 0.85791
Saving model ...
Model saved !  in  ../../Results/Models/LinearRe

KeyboardInterrupt: 